<a href="https://colab.research.google.com/github/rahadiyant/Data_AWS_Maritim/blob/main/AWS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit==1.27.0

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install scipy
!pip install plotly
!pip install openpyxl

In [ ]:
%%writefile aws3.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Judul aplikasi
st.title('Random Forest untuk Data AWS Maritim')

# Load data
@st.cache_data
def load_data():
    url = "https://github.com/rahadiyant/Data_AWS_Maritim/raw/main/1_thn_Data_AWS_Maritim.xlsx"
    return pd.read_excel(url)

data = load_data()

# Tampilkan data
st.subheader('Data Preview')
st.write(data.head())

# Pilih fitur dan target
st.sidebar.header('Pilih Fitur dan Target')
numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
features = st.sidebar.multiselect('Pilih fitur', numeric_columns)
target = st.sidebar.selectbox('Pilih target', numeric_columns)

if features and target:
    X = data[features]
    y = data[target]

    # Check if target is continuous or categorical
    is_continuous = len(np.unique(y)) > 10  # Arbitrary threshold, adjust as needed

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    n_estimators = st.sidebar.slider('Jumlah trees', 10, 200, 100)
    max_depth = st.sidebar.slider('Max depth', 1, 20, 5)

    if is_continuous:
        model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    else:
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)

    model.fit(X_train, y_train)

    # Prediksi
    y_pred = model.predict(X_test)

    # Evaluasi model
    if is_continuous:
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        st.subheader('Model Performance')
        st.write(f'Mean Squared Error: {mse:.2f}')
        st.write(f'R-squared: {r2:.2f}')
    else:
        accuracy = accuracy_score(y_test, y_pred)
        st.subheader('Model Accuracy')
        st.write(f'Accuracy: {accuracy:.2f}')
        st.subheader('Classification Report')
        st.text(classification_report(y_test, y_pred))

    # Feature importance
    st.subheader('Feature Importance')
    feature_importance = pd.DataFrame({'feature': features, 'importance': model.feature_importances_})
    feature_importance = feature_importance.sort_values('importance', ascending=False)
    st.bar_chart(feature_importance.set_index('feature'))

    # 2D Scatter Plot
    st.subheader('2D Scatter Plot')
    if len(features) >= 2:
        fig, ax = plt.subplots()
        scatter = ax.scatter(X[features[0]], X[features[1]], c=y, cmap='viridis')
        plt.colorbar(scatter)
        plt.xlabel(features[0])
        plt.ylabel(features[1])
        st.pyplot(fig)

    # 3D Scatter Plot
    st.subheader('3D Scatter Plot')
    if len(features) >= 3:
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        scatter = ax.scatter(X[features[0]], X[features[1]], X[features[2]], c=y, cmap='viridis')
        ax.set_xlabel(features[0])
        ax.set_ylabel(features[1])
        ax.set_zlabel(features[2])
        plt.colorbar(scatter)
        st.pyplot(fig)

    # Prediksi dengan input user
    st.subheader('Prediksi')
    user_input = {}
    for feature in features:
        user_input[feature] = st.number_input(f'Input {feature}', value=X[feature].mean())

    if st.button('Prediksi'):
        user_df = pd.DataFrame([user_input])
        prediction = model.predict(user_df)
        st.write(f'Prediksi: {prediction[0]}')

else:
    st.write('Silakan pilih fitur dan target di sidebar.')

In [ ]:
!streamlit run aws3.py & npx localtunnel --port 8501